In [ ]:
!pip install transformers

In [2]:
from tqdm.notebook import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
import random
import numpy as np

# Transformers and Torch
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AutoTokenizer, BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import f1_score

In [3]:
df = pd.read_csv("/content/drive/MyDrive/cleaned_data.csv", index_col=0)
df.head()

,medical_specialty,transcription,labels,transcription_cleaned_simple
0,Emergency Room Reports,"REASON FOR THE VISIT:, Very high PT/INR.,HIST...",0,reason visit high pt inr history patient year ...
1,Surgery,"PREOPERATIVE DIAGNOSIS:, Acetabular fracture ...",1,preoperative diagnosis acetabular fracture lef...
2,Surgery,"NAME OF PROCEDURE,1. Selective coronary angio...",1,name procedure selective coronary angiography ...
3,Radiology,"REFERRING DIAGNOSIS: , Motor neuron disease.,P...",2,referring diagnosis motor neuron disease perti...
4,Emergency Room Reports,"CHIEF COMPLAINT: , Dental pain.,HISTORY OF PRE...",0,chief complaint dental pain history present il...


In [4]:
# Split the dataset into training, validation, and testing sets
Train_X, Test_X, Train_y, Test_y = train_test_split(df['transcription_cleaned_simple'], df['labels'], test_size=0.3, random_state=42)

In [ ]:
# Initialize BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the training data
train_encodings = tokenizer.batch_encode_plus(
    Train_X, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    truncation=True, padding=True,
    return_tensors='pt'
)

# Tokenize the Validation data
test_encodings = tokenizer.batch_encode_plus(
    Test_X, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    truncation=True, padding=True,
    return_tensors='pt'
)


# Convert the tokenized inputs to PyTorch tensors
train_dataset = torch.utils.data.TensorDataset(torch.tensor(train_encodings['input_ids']),
                                              torch.tensor(train_encodings['attention_mask']),
                                              torch.tensor(Train_y.tolist()))

val_dataset = torch.utils.data.TensorDataset(torch.tensor(test_encodings['input_ids']),
                                            torch.tensor(test_encodings['attention_mask']),
                                            torch.tensor(Test_y.tolist()))


_____________


In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=40,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

In [8]:
batch_size = 3

dataloader_train = DataLoader(train_dataset, 
                              sampler=RandomSampler(train_dataset), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(val_dataset, 
                                   sampler=SequentialSampler(val_dataset), 
                                   batch_size=batch_size)

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

In [10]:
epochs = 5
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [19]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [12]:
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

cuda


In [14]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [15]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:
        model.zero_grad()
        batch = tuple(b.to(device) for b in batch)
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       
        outputs = model(**inputs)

        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'./finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/926 [00:00<?, ?it/s]


Epoch 1
Training loss: 2.717761167761574
Validation loss: 2.518637831925745
F1 Score (Weighted): 0.18336907820816184


Epoch 2:   0%|          | 0/926 [00:00<?, ?it/s]


Epoch 2
Training loss: 2.250514192921039
Validation loss: 2.228371187031119
F1 Score (Weighted): 0.22552396099988845


...


Epoch 3
Training loss: 1.9972956385906004
Validation loss: 2.127488840286918
F1 Score (Weighted): 0.24083006921260958


Epoch 4:   0%|          | 0/926 [00:00<?, ?it/s]


Epoch 4
Training loss: 1.8317829772475986
Validation loss: 2.0461664358975002
F1 Score (Weighted): 0.2588606313657632


Epoch 5:   0%|          | 0/926 [00:00<?, ?it/s]


Epoch 5
Training loss: 1.721611389556405
Validation loss: 2.04889366608723
F1 Score (Weighted): 0.25349258428576527


In [18]:
possible_labels = df.medical_specialty.unique()
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{' Emergency Room Reports': 0,
 ' Surgery': 1,
 ' Radiology': 2,
 ' Podiatry': 3,
 ' Neurology': 4,
 ' Gastroenterology': 5,
 ' Orthopedic': 6,
 ' Cardiovascular / Pulmonary': 7,
 ' Nephrology': 8,
 ' ENT - Otolaryngology': 9,
 ' General Medicine': 10,
 ' Hematology - Oncology': 11,
 ' Cosmetic / Plastic Surgery': 12,
 ' SOAP / Chart / Progress Notes': 13,
 ' Chiropractic': 14,
 ' Psychiatry / Psychology': 15,
 ' Consult - History and Phy.': 16,
 ' Hospice - Palliative Care': 17,
 ' Neurosurgery': 18,
 ' Obstetrics / Gynecology': 19,
 ' Urology': 20,
 ' Discharge Summary': 21,
 ' Autopsy': 22,
 ' Dermatology': 23,
 ' Letters': 24,
 ' Office Notes': 25,
 ' Lab Medicine - Pathology': 26,
 ' Ophthalmology': 27,
 ' Speech - Language': 28,
 ' Dentistry': 29,
 ' Pediatrics - Neonatal': 30,
 ' Physical Medicine - Rehab': 31,
 ' Bariatrics': 32,
 ' Endocrinology': 33,
 ' Pain Management': 34,
 ' IME-QME-Work Comp etc.': 35,
 ' Allergy / Immunology': 36,
 ' Sleep Medicine': 37,
 ' Diets and Nut

____


In [20]:
_, predictions, true_vals = evaluate(dataloader_validation)
accuracy_per_class(predictions, true_vals)


Class:  Emergency Room Reports
Accuracy: 0/22

Class:  Surgery
Accuracy: 156/221

Class:  Radiology
Accuracy: 16/58

Class:  Podiatry
Accuracy: 0/13

Class:  Neurology
Accuracy: 20/47

Class:  Gastroenterology
Accuracy: 11/56

Class:  Orthopedic
Accuracy: 31/89

Class:  Cardiovascular / Pulmonary
Accuracy: 28/94

Class:  Nephrology
Accuracy: 0/23

Class:  ENT - Otolaryngology
Accuracy: 1/35

Class:  General Medicine
Accuracy: 5/78

Class:  Hematology - Oncology
Accuracy: 0/21

Class:  Cosmetic / Plastic Surgery
Accuracy: 0/5

Class:  SOAP / Chart / Progress Notes
Accuracy: 10/33

Class:  Chiropractic
Accuracy: 0/5

Class:  Psychiatry / Psychology
Accuracy: 1/12

Class:  Consult - History and Phy.
Accuracy: 70/134

Class:  Hospice - Palliative Care
Accuracy: 0/1

Class:  Neurosurgery
Accuracy: 0/20

Class:  Obstetrics / Gynecology
Accuracy: 4/36

Class:  Urology
Accuracy: 8/45

Class:  Discharge Summary
Accuracy: 3/25

Class:  Autopsy
Accuracy: 0/2

Class:  Dermatology
Accuracy: 0/6

Cl

Reused code from: https://github.com/susanli2016/NLP-with-Python/blob/master/Text_Classification_With_BERT.ipynb